In [1]:
source('../common.r')

In [3]:
d <- data.rawmix(where="name like 'v0.28.1%' and nclients = 4 and duration = 30 and length = 4 and rate = 100")
d <- subset(d, commute_ratio == 0.5 & alpha == 0.6)

In [ ]:
sql("SELECT cc_ph, avg(throughput) FROM d WHERE nthreads = 96 GROUP BY cc_ph")

# Best Zipf

In [120]:
d <- data.rawmix(where="name like 'v0.28.1%' and nclients = 4 and duration = 30 and length = 4 and rate = 100 and timeout_scaling = 1000 and phase_limit = 100")
d$zipf <- num(d$alpha)
d <- subset(d, commute_ratio == 0.5)

In [121]:
unique(d$ccmode)

[1] "rw"     "simple" "better"

In [122]:
d.mean <- ddply(d, .(nthreads,cc_ph,zipf,phasing,cc), summarize, throughput=mean(throughput))
d.max <- ddply(d.mean, .(cc_ph,zipf), summarize, throughput=max(throughput))

In [125]:
{
  d.better <- subset(d.max, cc_ph == BETT+COMB+PH)
  d.notxn  <- subset(d.max, cc_ph == NOTXN)
  zipf.speedups <- d.better$throughput / d.notxn$throughput
}
mean(zipf.speedups)

[1] 0.8248753

In [23]:
q <- sql("SELECT zipf, cc_ph, throughput FROM `d.max` WHERE zipf = 1.4 ORDER BY zipf")
q

,zipf,cc_ph,throughput
1,1.4,non-transactional,10924.37
2,1.4,better boosting + combining + phasing,13555.6
3,1.4,better boosting + phasing,6724.25
4,1.4,boosting + combining + phasing,12459.71
5,1.4,boosting + phasing,6542.957
6,1.4,r/w locks + phasing,753.7955
7,1.4,boosting + combining,449.0317
8,1.4,boosting,1517.632
9,1.4,better boosting + combining,12841.4
10,1.4,better boosting,6668.132


## Speedups

In [56]:
speedups <- function(q) {
    speedup.base <- q[q$cc_ph == BETT+COMB+PH,]$throughput / q[q$cc_ph == RW+BASE,]$throughput
    speedup.nontxn <- q[q$cc_ph == BETT+COMB+PH,]$throughput / q[q$cc_ph == NOTXN,]$throughput
    cat("speedup over baseline:          " + round(speedup.base,2) + "\n")
    cat("speedup over non-transactional:  " + round(speedup.nontxn,2) + "\n")
}

In [57]:
speedups(q)

speedup over baseline:          48.84
speedup over non-transactional:  1.24


## Rawmix: Commute Ratio

In [101]:
d <- data.rawmix(where="name like 'v0.28.1%' and nclients = 4 and duration = 30 and length = 4 and rate = 100")
levels(d$phasing)

[1] "off" "on"

In [97]:
{
  d <- data.rawmix(where="name like 'v0.28.1%' and nclients = 4 and duration = 30 and length = 4 and rate = 100")
  d$zipf <- num(d$alpha)
  d.mix <- subset(d, nkeys == 1000 & zipf == 0.6)
  d <- subset(d.mix, select = c('rate', 'nthreads', 'cc_ph', 'cc', 'zipf', 'commute_ratio', 'phasing', 'timeout_scaling', 'throughput', 'op_timeouts'))
  d.mean <- ddply(d, .(nthreads,cc_ph,commute_ratio,phasing,cc), summarize, throughput=mean(throughput))
  d.max <- ddply(d.mean, .(cc_ph,phasing,cc,commute_ratio), summarize, throughput=max(throughput))
  d.mix <<- d.max
}

In [114]:
d.better <- subset(d.mix, cc_ph == BETT+COMB+PH)
d.notxn  <- subset(d.mix, cc_ph == NOTXN)

speedups <- d.better$throughput / d.notxn$throughput
speedups

[1] 0.7833409 0.7093968 0.6708936 0.6541169 0.6351011 0.6280014 0.6223550
 [8] 0.7610298 0.7110524 0.6718596 0.6522800 0.6449613 0.6230186 0.6297064

In [116]:
mean(speedups)

[1] 0.6712224

# Rubis

In [89]:
{
    d <- data.rubis(where="duration = 60 and name like 'v0.28.1%' and nthreads <= 96")
    d$workload <- revalue(d$mix, c(
      'mixed'     = 'read-heavy',
      'bid-heavy' = 'bid-heavy',
      'no-browse' = 'mostly bid'
    ))

    d.rubis <<- subset(d, lambda == 20 & grepl('read-heavy|bid-heavy', workload))
}

The following `from` values were not present in `x`: better#0, better#1
The following `from` values were not present in `x`: better#0, better#1
The following `from` values were not present in `x`: better#0#on, better#1#on


In [90]:
q.rubis <- sql("SELECT workload, cc_ph, avg(throughput) as throughput FROM `d.rubis` WHERE nthreads = 64 GROUP BY workload, cc_ph ORDER BY workload")
q.rubis

,workload,cc_ph,throughput
1,bid-heavy,boosting,12431.14
2,bid-heavy,boosting + combining,13431.78
3,bid-heavy,boosting + combining + phasing,13602.02
4,bid-heavy,boosting + phasing,13212.98
5,bid-heavy,r/w locks + phasing,10346.39
6,bid-heavy,r/w locks (baseline),198.8181
7,read-heavy,boosting,13184.86
8,read-heavy,boosting + combining,13486.95
9,read-heavy,boosting + combining + phasing,13987.95
10,read-heavy,boosting + phasing,13790.27


In [91]:
speedup <- function(q) {
  speedup.base <- q[q$cc_ph == COMB+PH,]$throughput / q[q$cc_ph == RW+BASE,]$throughput
  cat("speedup over baseline: " + round(speedup.base,2) + "\n")
}

cat("bid-heavy  -> ")
speedup(subset(q.rubis, grepl('bid-heavy', workload)))

cat("read-heavy -> ")
speedup(subset(q.rubis, grepl('read-heavy', workload)))

bid-heavy  -> speedup over baseline: 68.41
read-heavy -> speedup over baseline: 3.64


# Retwis

In [79]:
d.retwis <- data.retwis(where="name like '%v0.27.5%' and nclients = 2")
d.retwis <- subset(d.retwis, async == 0 & txn_failed < 20 & total_time > 60 & total_time < 65)

The following `from` values were not present in `x`: better#0, better#1
The following `from` values were not present in `x`: better#0, better#1
The following `from` values were not present in `x`: better#0#off#0, better#1#off#0, better#0#on#0, better#1#on#0, rw#0#off#1
The following `from` values were not present in `x`: simple:1
The following `from` values were not present in `x`: geom_heavy, update_heavy


In [80]:
unique(d.retwis$workload)

[1] post-heavy: ~45% (re)post read-heavy: ~14% (re)post
Levels: read-heavy: ~14% (re)post post-heavy: ~45% (re)post mixed

In [81]:
q.retwis <- sql("SELECT workload, cc_ph, avg(throughput) as throughput FROM `d.retwis` WHERE nthreads = 96 GROUP BY workload, cc_ph ORDER BY workload")
q.retwis

,workload,cc_ph,throughput
1,post-heavy: ~45% (re)post,boosting + combining,2817.13
2,post-heavy: ~45% (re)post,boosting + combining + phasing,8107.97
3,post-heavy: ~45% (re)post,boosting + phasing,7967.313
4,post-heavy: ~45% (re)post,r/w locks + phasing,1926.622
5,read-heavy: ~14% (re)post,boosting,8745.935
6,read-heavy: ~14% (re)post,boosting + combining,10530.22
7,read-heavy: ~14% (re)post,boosting + combining + phasing,16393.15
8,read-heavy: ~14% (re)post,boosting + phasing,15954.43
9,read-heavy: ~14% (re)post,r/w locks + phasing,7536.04
10,read-heavy: ~14% (re)post,r/w locks (baseline),6467.136


In [82]:
speedup <- function(q) {
  r.best <- max(subset(q, grepl('boosting', cc_ph))$throughput)
  r.base <- subset(q, cc_ph == RW+PH)$throughput
  cat("speedup over baseline: " + round(r.best / r.base, 2) + "\n")
}

cat("post-heavy -> ")
speedup(subset(q.retwis, grepl('post-heavy', workload)))

cat("read-heavy -> ")
speedup(subset(q.retwis, grepl('read-heavy', workload)))

post-heavy -> speedup over baseline: 4.21
read-heavy -> speedup over baseline: 2.18
